# Name : Samujjwaal Dey

## CS 582 Information Retrieval : Homework 1

In [6]:
import os
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup

In [2]:
#Importing the NLTK Stopwords
import nltk
nltk.download('stopwords')
stop_list = stopwords.words('english')
print(stop_list)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to C:\Users\Samujjwaal
[nltk_data]     Dey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [120]:
in_path = 'cranfieldDocs'
out_path = 'preprocessed_cranfieldDocs'
os.mkdir(out_path)
filenames = os.listdir(path)   #To generate file path
# print(filenames)

In [113]:
filepath = in_path + '/' + filenames[0]
file = open(filepath)
data = file.read()
file.close()

In [127]:
soup = BeautifulSoup(data)
title = soup.findAll('title')
text = soup.findAll('text')
# for item in title:
#     print(item.get_text())

In [119]:
# print(soup.prettify())

In [130]:
for fname in filenames:
    infilepath = in_path + '/' + fname
    outfilepath = out_path + '/' + fname
    with open(outfilepath, 'w') as outfile:    
        with open(infilepath) as infile:
            soup = BeautifulSoup(infile.read())
            title = soup.findAll('title')
            text = soup.findAll('text')
            for item in title:
                outfile.write(item.get_text())
            for item in text:
                 outfile.write(item.get_text())
        infile.close()
outfile.close()

In [131]:
filepath = out_path + '/' + filenames[0]
file = open(filepath)
data = file.read()
data

'\nexperimental investigation of the aerodynamics of a\nwing in a slipstream .\n\n  an experimental study of a wing in a propeller slipstream was\nmade in order to determine the spanwise distribution of the lift\nincrease due to slipstream at different angles of attack of the wing\nand at different free stream to slipstream velocity ratios .  the\nresults were intended in part as an evaluation basis for different\ntheoretical treatments of this problem .\n  the comparative span loading curves, together with supporting\nevidence, showed that a substantial part of the lift increment\nproduced by the slipstream was due to a /destalling/ or boundary-layer-control\neffect .  the integrated remaining lift increment,\nafter subtracting this destalling lift, was found to agree\nwell with a potential flow theory .\n  an empirical evaluation of the destalling effects was made for\nthe specific configuration of the experiment .\n'